In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [3]:
busstops = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/busstops_norway.csv')
grunnkrets_age = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_age_distribution.csv')
grunnkrets_household = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_income_households.csv')
grunnkrets_norway = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/plaace_hierarchy.csv')
sample_submission = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/sample_submission.csv')
stores_extra = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_extra.csv')
stores_test = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_test.csv')
stores_train = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_train.csv')

train_df = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/notebooks/Arash/submit_files/plaace_hiearchy_id_splitted.csv")


In [5]:
grunnkrets_household

,grunnkrets_id,year,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles
0,12010509,2015,0,0,4,0,0,0,0,4
1,14390206,2015,0,7,0,0,0,0,0,3
2,18040514,2015,10,0,0,0,0,0,0,7
3,18040804,2015,0,0,0,0,0,0,0,0
4,18200101,2015,0,6,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...
26004,3013310,2015,954,284,878,348,130,111,220,388
26005,20120606,2015,483,207,746,476,91,119,223,590
26006,2350801,2016,614,150,863,520,77,93,248,479
26007,3013317,2015,1088,270,886,388,173,103,250,553


In [136]:
def impude_NaN (stores_df, grunnkrets_df, NaN_string ): 
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    geo_df2 = geo_df.drop("year", axis = 1)
    merged_df = stores_df.merge(geo_df2, how = "left", on = "grunnkrets_id")
    NaN_df = merged_df[merged_df[NaN_string].isna()]
    split_df = merged_df[merged_df[NaN_string].notna()]

    mat = cdist(NaN_df[['lat', 'lon']],
                split_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(mat, index= NaN_df['grunnkrets_id'], columns=split_df['grunnkrets_id'])

    grunnkrets_id = NaN_df.grunnkrets_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    closest_df_with_distance = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values, "distance": distance.values})
    closest_df = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values})

    df_with_values_from_valid_id = split_df[split_df["grunnkrets_id"].isin(closest.values)]
    df_with_values_from_valid_id_removed_duplicates = df_with_values_from_valid_id.drop_duplicates(subset = ["grunnkrets_id"])

    df_valid_geo_data = df_with_values_from_valid_id_removed_duplicates.iloc[:,12:]
    df_with_only_gk_id = df_with_values_from_valid_id_removed_duplicates[["grunnkrets_id"]]
    df_list = [df_with_only_gk_id,df_valid_geo_data ]
    df_valid_geo_data_and_id = pd.concat(df_list, axis=1)

    df_without_nan = NaN_df.iloc[:,:12]

    df_including_closest_valid_id = df_without_nan.reset_index().merge(closest_df, how = "left", on = "grunnkrets_id").set_index("index")

    df_impuded = df_including_closest_valid_id.reset_index().merge(df_valid_geo_data_and_id, how="left", left_on ="closest_valid_id", right_on="grunnkrets_id").set_index("index")
    df_impuded_without_duplicates = df_impuded.drop_duplicates(subset="store_id")

    new_df_impuded = df_impuded_without_duplicates.drop(["closest_valid_id", "grunnkrets_id_y"], axis = 1).rename(columns= {"grunnkrets_id_x": "grunnkrets_id"})


    impuded = pd.concat([split_df, new_df_impuded])

    return impuded




In [137]:
def impude_NaN_test_set(test_df, grunnkrets_df, NaN_string): 
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    geo_df2 = geo_df.drop("year", axis = 1)
    merged_df = test_df.merge(geo_df2, how = "left", on = "grunnkrets_id")
    NaN_df = merged_df[merged_df[NaN_string].isna()]
    split_df = merged_df[merged_df[NaN_string].notna()]

    mat = cdist(NaN_df[['lat', 'lon']],
                split_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(mat, index= NaN_df['grunnkrets_id'], columns=split_df['grunnkrets_id'])

    grunnkrets_id = NaN_df.grunnkrets_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    closest_df_with_distance = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values, "distance": distance.values})
    closest_df = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values})

    df_with_values_from_valid_id = split_df[split_df["grunnkrets_id"].isin(closest.values)]
    df_with_values_from_valid_id_removed_duplicates = df_with_values_from_valid_id.drop_duplicates(subset = ["grunnkrets_id"])

    df_valid_geo_data = df_with_values_from_valid_id_removed_duplicates.iloc[:,11:]
    df_with_only_gk_id = df_with_values_from_valid_id_removed_duplicates[["grunnkrets_id"]]
    df_list = [df_with_only_gk_id,df_valid_geo_data ]
    df_valid_geo_data_and_id = pd.concat(df_list, axis=1)

    df_without_nan = NaN_df.iloc[:,:11]

    df_including_closest_valid_id = df_without_nan.reset_index().merge(closest_df, how = "left", on = "grunnkrets_id").set_index("index")

    df_impuded = df_including_closest_valid_id.reset_index().merge(df_valid_geo_data_and_id, how="left", left_on ="closest_valid_id", right_on="grunnkrets_id").set_index("index")
    df_impuded_without_duplicates = df_impuded.drop_duplicates(subset="store_id")

    new_df_impuded = df_impuded_without_duplicates.drop(["closest_valid_id", "grunnkrets_id_y"], axis = 1).rename(columns= {"grunnkrets_id_x": "grunnkrets_id"})


    impuded = pd.concat([split_df, new_df_impuded])

    return impuded

In [188]:
stores_gk_impuded = impude_NaN(stores_train, grunnkrets_norway, NaN_string="district_name")
stores_house_impuded = impude_NaN(stores_train, grunnkrets_household, "singles")
stores_age_impuded = impude_NaN(stores_train, grunnkrets_age, NaN_string="age_0")
stores_test_impuded = impude_NaN_test_set(stores_test, grunnkrets_norway, NaN_string="district_name")





In [189]:
stores_gk_impuded2 = stores_gk_impuded.set_index("store_id").drop(["geometry"], axis = 1)
stores_test_impuded2 = stores_test_impuded.set_index("store_id").drop(["geometry"], axis = 1)

In [191]:
import re

unique_chain_names = sorted([str(item) for item in set(stores_train['chain_name']).union(set(stores_test['chain_name'])).union(set(stores_extra['chain_name']))],key=len, reverse=True)

def chain_imputer(stores_df, unique_chain_names):
    return_tuples = []
    for index, row in stores_df.iterrows():
        store_id = row['store_id']
        chain_name = row['chain_name']
        store_name = row['store_name']
        if type(chain_name) is float:
            for _chain_name in unique_chain_names:
                re_chain_name = re.sub(r'[^A-Za-z0-9 ]+', '', str(_chain_name))
                re_store_name = re.sub(r'[^A-Za-z0-9 ]+', '', store_name)
                if re_chain_name in re_store_name[:len(re_chain_name)+1] and len(store_name.split(" ")[0]) == len(_chain_name.split(" ")[0]):
                    return_tuples.append((store_id, store_name, _chain_name))
                    break
    return pd.DataFrame(return_tuples, columns=['store_id', "store_name", 'chain_name'])




In [192]:
stores_chain_impuded = chain_imputer(stores_train, unique_chain_names=unique_chain_names)
stores_chain_test_impuded = chain_imputer(stores_test, unique_chain_names=unique_chain_names)

In [195]:
stores_gk_impuded2["chain_name"] = stores_gk_impuded2.chain_name.fillna(stores_chain_impuded.set_index("store_id").chain_name)
stores_test_impuded2["chain_name"] = stores_test_impuded2.chain_name.fillna(stores_chain_test_impuded.set_index("store_id").chain_name)

In [202]:
stores_gk_impuded2.to_csv("stores_gk_impuded.csv")
stores_test_impuded2.to_csv("stores_test_impuded.csv")

In [213]:
stores_gk_impuded2['t_district'] = stores_gk_impuded2['district_name'] + stores_gk_impuded2['municipality_name']

In [214]:
stores_gk_impuded2

,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,grunnkrets_name,district_name,municipality_name,area_km2,t_district
store_id,,,,,,,,,,,,,,,,
983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,Bragernes sentrum 3,Bragernes sentrum,Drammen,0.155779,Bragernes sentrumDrammen
987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,Sentrum 3 /rode 6,Sentrum 3,Oslo,0.264278,Sentrum 3Oslo
984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,Sydsiden 2,Hønefoss,Ringerike,0.160152,HønefossRingerike
914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,Sentrum 1,Sentrum,Bodø,0.095029,SentrumBodø
913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,Tiller-Hårstad 14,Tiller-Hårstad,Trondheim,0.251070,Tiller-HårstadTrondheim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898186512-971708093-15710,2016,JOKER VANNVÅG,2.8.1.0,Grocery stores,19360207,NaN,70.088196,19.993661,JOKER,NaN,16.842,Nord-Lenangen vest,Lenangen,Lyngen,0.848894,LenangenLyngen
996275094-971758961-1683,2016,NÆRBUTIKKEN RØDØY,2.8.1.0,Grocery stores,18360112,NaN,66.663386,13.080629,NÆRBUTIKKEN,NaN,8.517,Selsøyvik,Rødøy,Rødøy,1.236299,RødøyRødøy
952160427-971791772-29954,2016,JOKER WESTGAARD,2.8.1.0,Grocery stores,8150103,OLE HALVORSENS VEI 914,58.792096,9.372469,JOKER,NaN,20.777,Rørvik,Rørvik/Berg,Kragerø,0.504199,Rørvik/BergKragerø
